## Question 1: **Answers**

In [1]:
import sqlite3
import csv

In [ ]:
def load_crime_stats_data(input_data):
    try:
        # connecting to database and creating a cursor
        conn = sqlite3.connect("city_crime_database.db")
        cursor = conn.cursor()
        # Creating table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS crimes_data (
                city TEXT,
                year INT,
                violent_crime_rate FLOAT,
                property_crime_rate FLOAT
                )
        """ )
        with open(input_data, "r") as csv_file:
            csv_reader = csv.reader(csv_file)
            # Skip the header row
            next(csv_reader)
            for row in csv_reader:
                cursor.execute("INSERT INTO crimes_data VALUES (?, ?, ?, ?)", row)
        conn.commit()
        print("Data successfully saved in the database")
    except Exception as e:
        print(f"Error occured: {e}")
    finally:
        conn.close()

# Loading data
data = "/content/City_Crime_Rates.csv"
load_crime_stats_data(data)


Data successfully saved in the database


In [3]:
# Loading the extension
%load_ext sql

In [4]:
# Using the extension to connect to database
%sql sqlite:///city_crime_database.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM crimes_data
LIMIT 5;

 * sqlite:///city_crime_database.db
Done.


city,year,violent_crime_rate,property_crime_rate
New York City,2020,610.1,686.8
New York City,2021,588.8,574.1
Los Angeles,2020,572.2,184.8
Los Angeles,2021,546.5,926.8
Chicago,2020,648.3,320.7


## Questions 2: **Answer**

In [7]:
%%sql
-- Finding maximum rate
WITH max_rate AS (
    SELECT MAX(violent_crime_rate) AS max_rate
    FROM crimes_data
    WHERE Year = 2021
),
-- Finding minimum rate
min_rate AS (
    SELECT MIN(violent_crime_rate) AS min_rate
    FROM crimes_data
    WHERE Year = 2021
)
--Main query to return cities with min and max
SELECT DISTINCT
    City,
    Year,
    printf("%20s", (violent_crime_rate)) AS violent_crime_rate
FROM crimes_data
WHERE Year = 2021 AND violent_crime_rate = (SELECT max_rate FROM max_rate)
                  OR violent_crime_rate = (SELECT min_rate FROM min_rate);

 * sqlite:///city_crime_database.db
Done.


city,year,violent_crime_rate
Chicago,2021,611.5
San Jose,2021,266.2


## Questions 3: **Answer**

In [8]:
%%sql
-- Retrieving the minimum crime rate
WITH minimum_rate AS (
    SELECT MIN(property_crime_rate) AS min_rate
    FROM crimes_data
    WHERE Year = 2021
)
-- Main query to retrieve city with minimum rate
SELECT
    DISTINCT City,
    printf('%20s',property_crime_rate) AS property_crime_rate
FROM crimes_data
WHERE Year = 2021 AND property_crime_rate = (SELECT min_rate FROM minimum_rate);

 * sqlite:///city_crime_database.db
Done.


city,property_crime_rate
Fort Worth,3.7


## Questions 4: **Answer**

In [9]:
%%sql
WITH changes_in_crime_rates AS (
  SELECT
        City,
        Year,
        property_crime_rate - LAG(property_crime_rate)
        OVER (PARTITION BY City ORDER BY Year) AS crime_rate_change
  FROM crimes_data
),
-- Calculating largest crime rate change
max_change AS (
  SELECT
        MAX(crime_rate_change) AS max_change
  FROM changes_in_crime_rates
)
-- Selecting city with highest crime rate change
SELECT
    City,
    Year,
    printf('%20s',crime_rate_change) AS property_crime_rate_change
FROM changes_in_crime_rates
WHERE crime_rate_change = (SELECT max_change FROM max_change)
ORDER BY City, Year;

 * sqlite:///city_crime_database.db
Done.


City,Year,property_crime_rate_change
Los Angeles,2021,742.0


## Questions 5: **Answer**

In [10]:
%%sql
WITH crime_totals AS (
    SELECT
        Year,
        City,
        SUM(violent_crime_rate + property_crime_rate)
        AS total_city_crimes_2020,
        DENSE_RANK() OVER (ORDER BY violent_crime_rate + property_crime_rate DESC)
        AS rank_position
    FROM crimes_data
    WHERE Year = 2020
    GROUP BY Year, City
)
SELECT
    Year,
    City,
    total_city_crimes_2020,
    rank_position
FROM Crime_Totals
WHERE rank_position <= 5;

 * sqlite:///city_crime_database.db
Done.


Year,City,total_city_crimes_2020,rank_position
2020,San Antonio,1371.6,1
2020,Dallas,1308.0,2
2020,New York City,1296.9,3
2020,Phoenix,1184.7,4
2020,Houston,1058.6,5
